In [ ]:
#instrukce
#skript vyžaduje mít xml dokumenty na jednom místě
#na stejném místě mít i queries_cs
#cestu k těmto dokumentům je potřeba na jednom místě zadat
#tato proměnná se jmenuje "path"

In [ ]:
#1. nejdřív sebrat všechny dokumenty
#2. tokenizovat
#3. normalizovat
#4. udělat inverted index

In [4]:
#funkce pro konverzi stringů do tokenů v listech
def Convert(string):
    li = list(string.split(" "))
    return li

In [21]:
#funkce pro vytvoření inverted indexu
from collections import defaultdict
def create_index (data):
    index = defaultdict(list)
    for i, tokens in enumerate(data):
        for token in tokens:
             index[token].append(i)
    return index

In [ ]:
#zpracování dat
import html
import os, sys
from lxml import etree
import xml.etree.ElementTree as ET

kolekce = []
hashed = []
text_ready = []

#v tomto dictu je pořadí "DOC" jako key a "DOCID" jako value
doc_ID_dict = {}

# counters
counter_child = 0
counter_node = 0
counter_doc_ID = 0
counter_file = 0

path = "/home/mazurri/InfoRetrieval"
directory = os.listdir(path)

for file in directory:
    counter_child = 0
    counter_file = counter_file + 1
    test = os.path.splitext(file)
    #test, předpokládá se, že queries_cs bude ve stejné složce
    if test[0] == "queries_cs":
        continue      
    elif test[1] == ".xml":
        print(file)
        #kvůli erroru s unescaped characters "&"
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        for child in root:
            ### populování doc_ID_dictu pro identifikaci dokumentu
            doc_ID = child.find("DOCID").text
            doc_ID_dict[counter_doc_ID] = doc_ID
            #nejdřív append, potom inkrement
            counter_doc_ID = counter_doc_ID + 1
            
            for node in child:
                #prvni 3 nodes jsou vzdy stejne a neobsahuji text, který by bylo vhodné využít
                if counter_node < 3:
                    counter_node = counter_node + 1
                    continue
                else:
                    text = root[counter_child][counter_node].text
                    
                    counter_node = counter_node + 1

                    ### normalizace ###
                    #vše do lower case
                    text = text.lower()

                    #interpunkční znaménka
                    text = text.replace("\n", " ")
                    text = text.replace("?", " ")
                    text = text.replace("!", " ")
                    #tady u tečky možná by bylo dobré použít regulární výrazy aby to nesmazalo tečku co je u čísla
                    text = text.replace(".", " ")
                    #tady u tečky možná by bylo dobré použít regulární výrazy aby to nesmazalo čárku co je u čísla
                    text = text.replace(",", " ")
                    text = text.replace(" - ", " ")
                    
                    # tokenizace do listu podle předdefinované funkce
                    text = Convert(text)
                    
                    ### hashovaní stringů ###
                    for i in text:
                        hashed.append(hash(i))
                    
                    ### vytvoření setů pro eliminaci duplikátů ###
                    hashed_set = set(hashed)
                    hashed = []

                    ### přidání setu do kolekce ###
                    kolekce.append(hashed_set)
                    hashed_set = None
  
            counter_node = 0
            counter_child = counter_child + 1
    else:
        continue

In [ ]:
konzultace
-style guide python: různí lidé následují různé guides (zejména jména proměnných, komentování), 
jde i o to jestli to dělám pro sebe, nebo pro jiné
-dtd -> xml parser (etree), může se stát, že některé položky neodpovídají dtd (s tím parser má problém),
jsou to data z praxe, takhle to vypadlo, takže potřeba to nějak ošetřit
-část mého kódu kde dělám-> recover=TRUE, tak tam se možná řeší víc chyb než jen &
-dělat counter manuálně je low-level
-regulární výrazy v normalizaci ->dá se napsat do jednoho výrazu
-parsování dotazů (to je to co jsem dělal s queries), náš úkol
-NOT je unární operátor
-nahradit "AND NOT" například AND_NOT a pracovat s tím jako s jedním operátorem
-typická úloha pro princip "rozděl a panuj", měl bych list jednotlivých slov v dotaze, neřešil bych jestli je něco
operátor, nebo ne
-PAVEL AND PETR (3 tokeny), funkce "vyřeš dotaz", dva parametry, který ukazují (pointers) 1-3, vyřeš dotaz na
pozici 1 až 3, čímž vyřeší celý dotaz, jde zleva doprava, až narazí na operátor, tak rozdělí na sub problémy
vyřeší 1 a pak 3

In [ ]:
regex na nahrazení AND NOT jedním tokenem
napsat rekurzní funkci, která bude postupně volat sama sebe
operátor tím pádem bude vždy nadruhé pozici

In [22]:
inverted_index = create_index(kolekce)

In [ ]:
### INPUT ###

In [189]:
# counters

counter_query_ID = 0
counter_query = 0

query_ready = []
operatory = ["AND", "OR", "NOT"]

for file in directory:
    test = os.path.splitext(file)
    #print(test[0])
    if test[0] == "queries_cs":
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser = parser)
        root = tree.getroot()
        
        for child in root:
            ### pro pojmenování výsledného filu
            doc_NUM = child.find("num").text
            query = child.find("query").text            
            
            counter_query_ID = counter_query_ID + 1
            
            print("na začátku každé query by měl být counter roven nule a teď je")
            counter_query = 0
            print(counter_query)
            converted_query = Convert(query)
            #jedna query měla prázdný znak, házelo to error
            if "" in converted_query:
                converted_query.remove("")
            for i in converted_query:#Asi jít token po tokenu a ptát se jestli náhodou není AND NOT OR
                #pokud za AND, nebo OR následuje operátor, tak je chci sloučit ať se s tím líp pracuje
                print("už jsem uvnitř a teď je counter query")
                print(counter_query)
                print(converted_query)
                token = converted_query[counter_query]
                #jak je dlouhy ten converted_query?
                print("počet tokenů je")
                print(len(converted_query))
                operator = converted_query[counter_query]
                print(token)

                if token not in operatory:
                    token = token.lower
                    print("je to slovo")
                    inverted_index[hash(token)]
                    if counter_query == len(converted_query)-1:
                        posledni_slovo = True
                        print("je to poslední slovo")
                    else:
                        posledni_slovo = False
                        print("není to poslední slovo")
                    
                    active_token = token
                    if posledni_slovo == False:
                        if counter_query+3 <= len(converted_query):
                            if converted_query[counter_query+2] in operatory:
                                operator_combo = True
                                primary_operator = converted_query[counter_query+1]
                                secondary_operator = converted_query[counter_query+2]

                                next_token = converted_query[counter_query+3]
                                print(len(inverted_index[hash(token)]))
                                print(len(inverted_index[hash(next_token)]))
                            else:
                                operator_combo = False
                                primary_operator = converted_query[counter_query+1]
                                secondary_operator = None

                                next_token = converted_query[counter_query+2]
                        else:
                            operator_combo = False
                            primary_operator = converted_query[counter_query+1]
                            secondary_operator = None

                            next_token = converted_query[counter_query+2]
                    
                    else:
                        #vytvořit file, kam zapíšu výsledky
                        continue
                        
                counter_query = counter_query+1
    else:
        continue

na začátku každé query by měl být counter roven nule a teď je
0
už jsem uvnitř a teď je counter query
0
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
inflace
je to slovo
není to poslední slovo
už jsem uvnitř a teď je counter query
1
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
AND
už jsem uvnitř a teď je counter query
2
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
Euro
je to slovo
není to poslední slovo
0
1010
už jsem uvnitř a teď je counter query
3
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
AND
už jsem uvnitř a teď je counter query
4
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
NOT
už jsem uvnitř a teď je counter query
5
['inflace', 'AND', 'Euro', 'AND', 'NOT', 'fotbal']
počet tokenů je
6
fotbal
je to slovo
je to poslední slovo
na začátku každé query by měl být counter roven nule a teď je
0
už jsem uvnitř a teď je counter query
0
['obnovitelný', 'AND', 'zdroj']
poče